<center>
<strong>
Optimización y Simlaciones
<br>
Punto 1 
</strong>
</center>

___

<center>
<strong>
Grupo 1
<br>
Cycle Cancelling
</strong>
</center>

___

*El algoritmo debe ser ejecutado de manera secuencial*

## Librerías necesarias

In [3]:
import pandas as pd
import sys
import pprint
import time

## Creación de grafo

In [4]:
matrizcsvcapacidades = pd.read_csv("Matrices en CSV/capacidadescanciclos.csv", sep = ";", encoding = 'utf-8', header = None)
grafoCapacidades = matrizcsvcapacidades.to_numpy()
numeroVertices = len(grafoCapacidades)

In [5]:
grafoCapacidades

array([[ 0, 50,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0],
       ...,
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ..., 25,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0]])

#### Matriz de costos

In [6]:
matrizcsvcostos = pd.read_csv("Matrices en CSV/costoscanciclos.csv", sep = ";", encoding = 'utf-8', header = None)
grafoCostos = matrizcsvcostos.to_numpy()

In [7]:
grafoCostos

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.20142092, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

## Métodos

*Comprabamos la relación entre el nodo origen y el nodo destino, si no hay, paramos el algo*

In [21]:
def bfs(origen, destino, grafoCapacidades, resgrafoCapacidades, numeroVertices):
  queue = [origen]
  paths = {origen: []}
  while queue:
    u = queue.pop(0)
    for v in range(numeroVertices):
      if grafoCapacidades[u][v] != 0:
        if grafoCapacidades[u][v] - resgrafoCapacidades[u][v] > 0 and v not in paths:
          paths[v] = paths[u] + [(u, v)]
          if v == destino:
            # print("Possible camino from ", origen, " to ", destinoino, "->", paths[v])
            return paths[v]
          queue.append(v)
  return None

*calculamos el flujo máximo con Edmonds Karp. Iterativamente sacamos el grafo residual en costos y capacidades*

In [16]:
def edmondsKarp(origen, destino, capacidad, costo, numeroVertices):
  resgrafoCapacidades = [[0] * numeroVertices for _ in range(numeroVertices)]
  resgrafoCosto = [[0] * numeroVertices for _ in range(numeroVertices)]

  while True:
    camino = bfs(origen, destino, capacidad, resgrafoCapacidades, numeroVertices)
    if not camino:
      break
        
    flujo = min(capacidad[u][v] - resgrafoCapacidades[u][v] for u, v in camino)
    # print("flujo:\n", flujo)
    
    for u, v in camino:
      resgrafoCapacidades[u][v] += flujo
      resgrafoCosto[u][v] = costo[u][v]
      resgrafoCapacidades[v][u] -= flujo
      resgrafoCosto[v][u] = costo[u][v]*-1

  return (resgrafoCapacidades, resgrafoCosto)

*Indicamos si hay ciclos negativos en el grafo con el algoritmo Bellman Ford*

In [17]:
def bellmanFord(graph, origen, numeroVertices):
  distancias = [sys.maxsize for i in range(numeroVertices)]
  prev = [-1 for i in range(numeroVertices)]
  distancias[origen] = 0

  for i in range(numeroVertices - 1):
    for u in range(numeroVertices):
        for v in graph[u].keys():
          distanciasTemporales = sys.maxsize
          
          if distancias[u] != sys.maxsize:
            distanciasTemporales = distancias[u] + graph[u][v][1]
          if distancias[v] > distanciasTemporales:
            distancias[v] = distanciasTemporales
            prev[v] = u

  ciclo = []
  for u in range(numeroVertices):
    for v in graph[u].keys():
      if distancias[v] > distancias[u] + graph[u][v][1]:
        C = v
        for i in range(numeroVertices):
          C = prev[C]
        v = C
        # temp = v
        while True:
          ciclo.append(v)
          if v == C and len(ciclo) > 1:
              break
          v = prev[v]
        ciclo.reverse()
        final_cycle = []
        for index in range(len(ciclo) - 1):
          final_cycle.append((ciclo[index], ciclo[index + 1]))
            
        # print("\nNegative ciclo:\n", final_cycle)
        return final_cycle
        
  return None


*Cycle cancelling para matriz de adyacencia*

In [18]:
def cycleCancelling(origen, destino, grafoCapacidades, grafoCostos, numeroVertices):
    resgrafoCapacidades, resgrafoCosto = edmondsKarp(origen, destino, grafoCapacidades, grafoCostos, numeroVertices)

    # Ponemos el valor restante para cada arista
    for u in range(numeroVertices):
      for v in range(len(grafoCapacidades)):
        if grafoCapacidades[u][v] != 0:
          if resgrafoCapacidades[u][v] > 0:
            if u == destino + 1:
              resgrafoCapacidades[u][v] = grafoCapacidades[u][v] - resgrafoCapacidades[u][v]
              resgrafoCosto[u][v] = 0
            else:
              resgrafoCapacidades[u][v] = grafoCapacidades[u][v] - resgrafoCapacidades[u][v]
              resgrafoCosto[u][v] = grafoCostos[u][v]
                        

    # Añadimos las aristas no utilizadas al grafo residual
    for u in range(numeroVertices):
      for v in range(len(grafoCapacidades)):
        if grafoCapacidades[u][v] != 0:
          if resgrafoCapacidades[v][u] == 0 and resgrafoCosto[v][u] == 0:
            resgrafoCapacidades[u][v] = grafoCapacidades[u][v]
            resgrafoCosto[u][v] = grafoCostos[u][v]

    # Convertimos las matrices residuales en una lista de diccionarios
    # Asi, podemos leerlos con el algoritmo de Bellman Ford
    grafoResidual = [{} for i in range(numeroVertices)]
    for u in range(numeroVertices):
      for v in range(numeroVertices):
        if resgrafoCapacidades[u][v] != 0 or resgrafoCosto[u][v] != 0:
          grafoResidual[u][v] = (abs(resgrafoCapacidades[u][v]), resgrafoCosto[u][v])
    # print("\nResidual Graph: ")
    # pprint.pprint(grafoResidual)

    while True:
      ciclo = bellmanFord(grafoResidual, destino, numeroVertices)
      if not ciclo:
        break
          
      flujo = min(grafoResidual[u][v][0] for u, v in ciclo)
      for u, v in ciclo:
        temp = grafoResidual[u][v]
        if u in grafoResidual[v].keys():
          grafoResidual[v][u] = (grafoResidual[v][u][0] + flujo, grafoResidual[v][u][1])
        else:
          grafoResidual[v][u] = (flujo, -grafoResidual[u][v][1])
        if temp[0] == flujo:
          del grafoResidual[u][v]
        else:
          grafoResidual[u][v] = (grafoResidual[u][v][0] - flujo, grafoResidual[u][v][1])
          temp = grafoResidual[u][v]

    costoFinal = 0
    # print("Final flujo is: ")
    for u in range(numeroVertices):
      for v in grafoResidual[u].keys():
        edge = grafoResidual[u][v]
        if edge[1] < 0:
          #print((v, u), ":", (edge[0], abs(edge[1])))
          print("-> Del nodo", v, "al nodo", u, "se envía un tráfico de", edge[0], "con un retardo de", abs(edge[1]))
          costoFinal += (edge[0]) * (abs(edge[1]))

    return costoFinal


*Go*

In [22]:
origen = 0
destino = numeroVertices - 1
print("Ruta óptima:")
#print("(Nodo del que sale, nodo al que llega) : (Tráfico que se envía, Retardo)")
costoFinal = cycleCancelling(origen, destino, grafoCapacidades, grafoCostos, numeroVertices)
print("\nRetardo mínimo = ", costoFinal)

Ruta óptima:
-> Del nodo 1 al nodo 5 se envía un tráfico de 9 con un retardo de 0.192245058
-> Del nodo 9 al nodo 8 se envía un tráfico de 6 con un retardo de 0.122244401
-> Del nodo 1 al nodo 8 se envía un tráfico de 2 con un retardo de 0.374331887
-> Del nodo 1 al nodo 9 se envía un tráfico de 39 con un retardo de 0.043189676
-> Del nodo 5 al nodo 13 se envía un tráfico de 9 con un retardo de 0.301797001
-> Del nodo 9 al nodo 13 se envía un tráfico de 17 con un retardo de 0.445052009
-> Del nodo 8 al nodo 13 se envía un tráfico de 8 con un retardo de 0.25593562
-> Del nodo 9 al nodo 14 se envía un tráfico de 16 con un retardo de 0.314895318
-> Del nodo 13 al nodo 18 se envía un tráfico de 21 con un retardo de 0.163697885
-> Del nodo 18 al nodo 19 se envía un tráfico de 8 con un retardo de 0.12868042
-> Del nodo 13 al nodo 22 se envía un tráfico de 13 con un retardo de 0.16257214
-> Del nodo 14 al nodo 22 se envía un tráfico de 16 con un retardo de 0.156101813
-> Del nodo 18 al nodo 2

*Algoritmo hecho con base a https://github.com/scheng95/cs51finalproject/blob/master/to_submit/code/4cycle_cancel.py, muchas gracias señores :')*